<p align="center">
  <img src="IPCA.png" alt="Logo IPCA" width="350" style="margin-bottom: 20px;"/>
</p>

<h1 align="center" style="font-size: 36px; margin-bottom: 10px;">Trabalho Prático 2</h1>

<h3 align="center" style="color: #cccccc; font-weight: normal; margin-top: 0;">Unidade Curricular de Inteligência Artificial</h3>

<br/>

<table align="center" style="border-collapse: collapse; width: 25%;">
  <tr>
    <th style="border: 1px solid #ccc; padding: 8px; background-color: #222;">Nomes</th>
    <th style="border: 1px solid #ccc; padding: 8px; background-color: #222;">Números</th>
  </tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Nuno Silva</td><td style="border: 1px solid #ccc; padding: 8px;">28005</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Joel Faria</td><td style="border: 1px solid #ccc; padding: 8px;">28001</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Diogo Graça</td><td style="border: 1px solid #ccc; padding: 8px;">28004</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Gonçalo Gomes</td><td style="border: 1px solid #ccc; padding: 8px;">25455</td></tr>
  <tr><td style="border: 1px solid #ccc; padding: 8px;">Hugo Monteiro</td><td style="border: 1px solid #ccc; padding: 8px;">27993</td></tr>
</table>

<br/>

<p align="center" style="font-size: 16px; margin-top: 20px;">
  <b>Docente:</b> Rui Fernandes<br/>
  <b>Data:</b> 12-12-2025
</p>

<hr style="width: 60%; border: 1px solid #555;"/>
<p align="center" style="font-size: 14px; color: #888;">
  Instituto Politécnico do Cávado e do Ave — Engenharia de Sistemas Informáticos
</p>


# Bibliotecas

In [ ]:
# Import required libraries

# linear algebra and data processing libraries
import numpy as np
import pandas as pd

# scikit-learn libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Graphics Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Utils
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Preparação e Limpeza dos Dados

Nesta etapa, realizamos o processamento inicial do dataset `boardgames.csv`:

1.  **Carregamento:** Leitura dos dados brutos.
2.  **Seleção de Features:** Filtragem das colunas relevantes para a análise.
3.  **Engenharia de Atributos:** Criação da variável `rating_category` para classificar os jogos baseados na nota média (*average*).

In [ ]:
# Load dataset

games_df = pd.read_csv('boardgames.csv')

#Filtrar colunas desnecessárias

cols_to_keep = [
    'primary',
    'yearpublished',
    'minplayers',
    'maxplayers',
    'minplaytime',
    'minage',
    'boardgamecategory',
    'boardgamemechanic',
    'boardgamefamily',
    'boardgamedesigner',
    'boardgameartist',
    'boardgamepublisher',
    'usersrated',
    'average'
]

games_df = games_df[cols_to_keep]

# Definição dos Novos Limites (Bins) e Rótulos (Labels)
bins = [0, 4, 6.2, 7.5, 10.1]
labels = ['bad', 'mediocre', 'good', 'excelent']

# Criação da nova coluna 'rating_category'
games_df['rating_category'] = pd.cut(
    games_df['average'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

print("Nr. rows - train: ", len(games_df))

# Visualização Inicial dos Dados

Vamos inspecionar as primeiras linhas do DataFrame para verificar se o carregamento ocorreu como esperado e entender a estrutura das colunas disponíveis.

In [ ]:
# Print top examples
games_df.head()

# Análise Estrutural e Estatística

Nesta etapa, utilizamos dois comandos fundamentais do Pandas para conhecer melhor os dados:

1.  **Estrutura (`info`):** Para verificar os tipos de dados (int, float, object), a quantidade de memória usada e se existem valores nulos (missing values).
2.  **Estatísticas (`describe`):** Para obter um resumo estatístico das variáveis numéricas, permitindo identificar rapidamente a escala dos dados, médias e possíveis anomalias (outliers).

In [ ]:
# Display basic information about the DataFrame
print(games_df.info())

# Display descriptive statistics for numerical features
print(games_df.describe())

# Display unique values for categorical features
# for column in games_df.columns:
#   if games_df[column].dtype == object:
#     print(f"\nUnique values for {column}:")
#     print(games_df[column].unique())

# Identificação de Variáveis Categóricas

Para prosseguir com a análise exploratória, precisamos separar as variáveis quantitativas (numéricas) das qualitativas (categóricas).

O código abaixo utiliza uma *list comprehension* para percorrer todas as colunas e selecionar apenas aquelas cujo tipo de dado (`dtype`) seja:
* `object`: Geralmente texto (strings).
* `category`: Tipo de dado específico do Pandas para categorias finitas.

In [ ]:
# Get the categorical variables from games_df
categorical_features = [
    feature 
    for feature in games_df.columns 
    if games_df[feature].dtype in ['object', 'category']
]
print(categorical_features)

# Função de Visualização: Gráfico de Barras

Para facilitar a Análise Univariada das variáveis categóricas, definimos a função `bar_plot`.
Ela realiza duas tarefas para cada variável enviada:
1.  Calcula a frequência de cada categoria.
2.  Gera um gráfico de barras e imprime os valores numéricos.

In [ ]:
def bar_plot(variable):
    """
        input: variable ex: "Sex"
        output: bar plot & value count
    """
    # get feature
    var = games_df[variable]
    # count number of categorical variable(value/sample)
    varValue = var.value_counts()

    # visualize
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
# Plot categorial_features with less than 10 distinct values
for cf in categorical_features:
  if games_df[cf].nunique() < 10:
    bar_plot(cf)

In [ ]:
# print values for categorial_features with more than 9 distinct values
for cf in categorical_features:
  if games_df[cf].nunique() >= 10:
    print("{} \n".format(games_df[cf].value_counts()))

# Visualização de Distribuição: Histogramas

O código a seguir realiza três passos principais:
1.  **Define uma função (`plot_hist`)**: Automatiza a criação de histogramas padronizados.
2.  **Identifica as colunas numéricas**: Filtra automaticamente todas as colunas que contêm números (excluindo texto e categorias).
3.  **Gera os gráficos**: Itera sobre a lista de colunas numéricas (exceto 'yearpublished', que será analisada separadamente) e exibe a distribuição de cada uma.

Isso nos permite identificar padrões como:
* A maioria dos jogos exige um número mínimo de jogadores (`minplayers`) baixo ou alto?
* Qual é a faixa de tempo de jogo (`minplaytime`) mais comum?
* Como as notas médias (`average`) estão distribuídas? Elas seguem uma curva normal?

In [ ]:
# Visualize frequency and distribution of numerical features
def plot_hist(variable):
    plt.figure(figsize = (9,3))
    plt.hist(games_df[variable], bins = 50)
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} distribution with hist".format(variable))
    plt.show()

In [ ]:
# Get the numerical variables from games_df
numerical_features = [feature for feature in games_df.columns if games_df[feature].dtype not in [object, 'category']]
print(numerical_features)

In [ ]:
removed_nf = ["yearpublished"]
for nf in numerical_features:
  if nf not in removed_nf:
    plot_hist(nf)

# Análise de Dados Ausentes: Idade Mínima

A coluna `minage` (idade mínima recomendada) possui dois tipos de dados que consideramos "ausentes" ou inválidos para esta análise:
1.  **Valores Nulos (NaN):** Campos vazios.
2.  **Valor Zero (0):** Indica que a idade não foi informada corretamente.

O código abaixo calcula o total desses casos e exibe exemplos para inspeção.

In [ ]:
# find age null values
print(f'Age has {games_df["minage"].isnull().sum() + (games_df["minage"] == 0).sum()} null values')

# Filtro para ver TODAS as linhas que você contou no total_missing
missing_rows = games_df[
    (games_df["minage"].isnull()) | 
    (games_df["minage"] == 0)
]

missing_rows.head()

# Análise Bivariada: Relacionamentos com a Nota (Rating)

Para entender melhor o que influencia a qualidade percebida de um jogo, vamos cruzar algumas variáveis importantes com a nossa categoria de avaliação (`rating_category`).

Utilizaremos **tabelas de contingência** (crosstabs) para analisar as frequências cruzadas entre:
1.  **Min Players:** Será que jogos projetados para um número específico de jogadores (ex: 2) têm melhor desempenho
2.  **Users Rated:** A popularidade (número de votos) está ligada à qualidade
3.  **Category:** Certos gêneros de jogos (ex: Wargames) tendem a receber notas mais altas
4.  **Publisher:** Algumas editoras são sinônimo de qualidade

In [ ]:
# Min Players vs Rating Category
print("\nMin Players vs Rating Category")
display(pd.crosstab(games_df['minplayers'], games_df['rating_category']))

# Quantidade de votos vs Rating Category
print("\nQty votes vs Rating Category")
display(pd.crosstab(games_df['usersrated'], games_df['rating_category']))

# Categoria do jogo vs Categoria de rating
print("\nBoard Game Category vs Rating Category")
display(pd.crosstab(games_df['boardgamecategory'], games_df['rating_category']))

# Publisher vs Categoria de rating
print("\nBoard Game Publisher vs Rating Category")
display(pd.crosstab(games_df['boardgamepublisher'], games_df['rating_category']))

# Visualização

In [ ]:
# Correlation Between Sibsp -- Parch -- Age -- Fare -- Survived
list1 = ["yearpublished" , "minage", "usersrated", "average"]
sns.heatmap(games_df[list1].corr(), annot = True, fmt = ".2f")
plt.show()

In [ ]:
# MinPlayers → Rating Category
g = sns.catplot(x="minplayers", y="average", data=games_df, kind="bar", height=6)
g.set_ylabels("Average Rating")
plt.show()

In [ ]:
# MinAge → Distribuição de Ratings
g = sns.FacetGrid(games_df, col="rating_category")
g.map(sns.distplot, "minage", bins=20)
plt.show()


In [ ]:
# Players (minplayers) × MinAge × Rating Category
g = sns.FacetGrid(games_df, col="rating_category", row="minplayers", aspect=1.5)
g.map(plt.hist, "minage", bins=20)
g.add_legend()
plt.show()

In [ ]:
# Year Published × Average Rating × Players
g = sns.FacetGrid(games_df, row="minplayers", aspect=6)
g.map(sns.pointplot, "yearpublished", "average")
g.add_legend()
plt.show()

In [ ]:
# MinAge vs MinPlayers
sns.catplot(x="minage",
            y="minplayers",
            hue="rating_category",
            data=games_df,
            kind="box",
            aspect=3
            )
plt.show()


# Tratamento de Valores Ausentes (Missing Values) e Visualização

Nesta etapa, focamos na limpeza e imputação de dados.
1.  **Identificação:** Localizamos quais colunas possuem valores nulos.
2.  **Visualização:** Utilizamos um *boxplot* para entender a distribuição de votos (`usersrated`) e identificar *outliers*.
3.  **Preenchimento:**
    * Para `boardgamemechanic` (categórico), preenchemos com "Unknown".
    * Para `usersrated` (numérico), preenchemos com a média da editora (`boardgamepublisher`), uma estratégia mais inteligente do que usar a média global.

In [ ]:
# get variables with null values
games_df.columns[games_df.isnull().any()]

In [ ]:
# list number of null values per variable
games_df.isnull().sum()

In [ ]:
games_df[games_df["boardgamemechanic"].isnull()]

In [ ]:
# check the fare distribution for ports
games_df.boxplot(column="usersrated",by = "rating_category")
plt.show()

In [ ]:
games_df[games_df["boardgamemechanic"].isnull()]

In [ ]:
# Assign the port to null embarked values according to the data: fare value
games_df["boardgamemechanic"] = games_df["boardgamemechanic"].fillna("Unknown")
games_df[games_df["boardgamemechanic"].isnull()]

In [ ]:
# Assign the fare overall average to the two null values in fare
publisher_means = games_df.groupby("boardgamepublisher")["usersrated"].mean()
usersrated_Unkown = games_df[games_df['usersrated'].isnull()].index
games_df["usersrated"] = games_df["usersrated"].fillna( games_df["boardgamepublisher"].map(publisher_means))

In [ ]:
# Show the "games_df" row for usersrated 1044

for idx in usersrated_Unkown:
    row = games_df.loc[idx]
    print("Nome:", row["primary"], "| Publisher:", row["boardgamepublisher"],
          "| Usersrated preenchido:", row["usersrated"])

# Engenharia de Atributos: Extração de Franquias

Nesta etapa, criamos uma nova variável para tentar capturar o "efeito da marca" ou franquia.
A lógica aplicada é:
1.  **Extrair Prefixo:** Identificamos a primeira palavra do nome do jogo. Se for uma palavra muito curta (menos de 5 letras), pegamos também a segunda palavra para dar mais contexto (ex: "The Resistance" vs apenas "The").
2.  **Desambiguação:** Como jogos diferentes podem começar com a mesma palavra, adicionamos o nome da editora (`boardgamepublisher`) para criar um identificador único de franquia (ex: "Catan (Kosmos)").

In [ ]:
# Typical names
games_df["boardgamepublisher"].head(10)


In [ ]:
# Extrair fraquia do nome do jogo
def extract_prefix(name):
    words = str(name).lower().split()
    
    if len(words) == 0:
        return ""
    
    # Primeira palavra
    w1 = words[0]
    
    # Se a primeira palavra tiver menos de 5 letras → usar 2 palavras (se existir) (The X e The Y podem ser franquias diferentes mas têm o mesmo prefixo)
    if len(w1) < 5 and len(words) > 1:
        return w1 + " " + words[1]
    
    # Caso contrário → usar só a primeira
    return w1

games_df["serie"] = games_df["primary"].apply(extract_prefix)

#Distingir franquias com o mesmo nome mas publisher diferente
games_df["franchise_name"] = (
    games_df["serie"].str.title() + 
    " (" + games_df["boardgamepublisher"].astype(str) + ")"
)

games_df["franchise_name"].head(10)

# Visualização: Top 5 Franquias com Mais Jogos

Agora que criamos a variável `franchise_name`, podemos visualizar quais são as "famílias" de jogos mais prolíficas no dataset.

O código abaixo gera um **gráfico de contagem (countplot)** horizontal. A parte mais importante é o argumento `order`, que garante que apenas as 5 franquias mais frequentes apareçam no gráfico, ordenadas da maior para a menor.

In [ ]:
#Top 5 franquias com mais jogos

sns.countplot(
    y="franchise_name",
    data=games_df,
    order=games_df["franchise_name"].value_counts().head(5).index
)
plt.xticks(rotation=0)
plt.show()


# Visualização: Média de Avaliação das Top Franquias

Além de saber quais franquias têm mais jogos, é crucial entender como elas são avaliadas pela comunidade. Quantidade não é qualidade!

O código abaixo realiza os seguintes passos:
1.  Identifica as **Top 5 franquias** com maior número de títulos lançados.
2.  Filtra o dataset para manter apenas os jogos dessas franquias.
3.  Gera um **gráfico de barras (catplot)** que mostra a nota média (`average`) de cada uma dessas franquias, permitindo comparar qual delas mantém o padrão de qualidade mais alto.

In [ ]:
#Average Review Score das Top 5 Franquias com mais jogos

top_x = 5  # número de franquias que queremos mostrar

# Contar quantos jogos cada franquia tem
franchise_counts = games_df["franchise_name"].value_counts()

# Selecionar as top 5 franquias com mais jogos
top_franchises = franchise_counts.head(top_x).index

# Filtrar o DataFrame para essas franquias
df_top = games_df[games_df["franchise_name"].isin(top_franchises)]

# Criar gráfico de barra mostrando a média dos reviews por franquia
g = sns.catplot(
    y="franchise_name",
    x="average",
    data=df_top,
    kind="bar",
    order=top_franchises,
    aspect=3
)

# Rotacionar labels para vertical
g.set_xticklabels(rotation=0)

# Renomear eixo Y
g.set_ylabels("Average Review Score of the top franchises")

plt.show()

In [ ]:
# One-hot encoding title
games_df = pd.get_dummies(games_df,columns=["rating_category"])
games_df.head()

# Detecção de Outliers (Método IQR)

Outliers são valores extremos que se desviam drasticamente da média dos dados e podem prejudicar a performance de modelos de Machine Learning.

O código abaixo define uma função `detect_outliers` que:
1.  Calcula o **IQR** (Diferença entre o 3º e o 1º quartil) para cada coluna.
2.  Define um limite de corte (geralmente 1.5 vezes o IQR).
3.  Identifica índices de linhas que excedem esses limites.
4.  **Filtro de Robustez:** Retorna apenas os índices que são considerados outliers em **mais de 2 colunas** diferentes, evitando descartar dados que são anômalos em apenas um aspecto.

In [ ]:
# function for finding outliers
def detect_outliers(df,features):
    outlier_indices = []

    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)

    # Define as multiplie outlier those indices that are outlier in more than two features
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)

    return multiple_outliers

In [ ]:
# find outliers
games_df.loc[detect_outliers(games_df, ["yearpublished", "minplayers", "maxplayers", "minplaytime", "minage" ,"usersrated", "average"])]

In [ ]:
# drop outliers
# games_df = games_df.drop(detect_outliers(games_df,["Age","SibSp","Parch","Fare"]),axis = 0).reset_index(drop = True)

# Detecção de Outliers Multidimensional: DBSCAN

O método IQR analisa cada variável isoladamente. No entanto, um jogo pode ter valores "normais" em cada característica individualmente, mas uma combinação "anormal" (ex: um jogo de 5 minutos que custa $200).

Para capturar esses casos, usamos o **DBSCAN**. Ele agrupa dados baseados na densidade e classifica pontos isolados como outliers (ruído).
1.  **Pré-processamento:** Selecionamos as variáveis numéricas e, crucialmente, aplicamos a **padronização (StandardScaler)**, pois o DBSCAN é sensível à escala dos dados.
2.  **Clusterização:** Rodamos o algoritmo. Pontos que não pertencem a nenhum cluster recebem o rótulo `-1`.
3.  **Resultado:** Filtramos e exibimos esses pontos anômalos.

In [ ]:
# Select numerical features and handle missing values, then scale the data using `StandardScaler`.
selected_features = ["yearpublished", "minplayers", "maxplayers", "minplaytime", "minage" ,"usersrated", "average"]
df_dbscan = games_df[selected_features].copy()

# Impute missing 'Age' values with the median
median_age = df_dbscan['average'].median()
df_dbscan['average'].fillna(median_age, inplace=True)

# Instantiate and apply StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_dbscan)
print("DataFrame for DBSCAN created, 'average' missing values imputed, and features scaled.")
print("Shape of scaled_features:", scaled_features.shape)


In [ ]:
# original parameters: eps=0.5, min_samples=5
dbscan = DBSCAN(eps=2, min_samples=3)
clusters = dbscan.fit_predict(scaled_features)

# Identify outliers (cluster label -1)
outliers_indices = df_dbscan[clusters == -1].index

# print the number of outliers found
print(f"Number of outliers detected by DBSCAN: {len(outliers_indices)}")

# print the 10 first outliers
if not outliers_indices.empty:
    display(games_df.loc[outliers_indices].head(10))
else:
    print("No outliers detected with current DBSCAN parameters.")

# Bibliografia

* **NumPy:** [Documentação Oficial](https://numpy.org/doc/)
* **Pandas:** [Documentação Oficial](https://pandas.pydata.org/docs/)
* **Scikit-learn:** [Guia do Usuário](https://scikit-learn.org/stable/user_guide.html)
* **Matplotlib:** [Galeria de Exemplos](https://matplotlib.org/stable/gallery/index.html)
* **Seaborn:** [Tutoriais](https://seaborn.pydata.org/tutorial.html)
* **IPython:** [Documentação](https://ipython.readthedocs.io/en/stable/)
* **Python Standard Library:** [Collections](https://docs.python.org/3/library/collections.html) & [Warnings](https://docs.python.org/3/library/warnings.html)
